In [1]:
import pandas as pd
import random
from sklearn.neighbors import KNeighborsClassifier

In [2]:
#Hasta 15 de Abril (para entrenar)

postulantes_edu_df = pd.read_csv('fiuba_1_postulantes_educacion_h15.csv')
postulantes_gye_df = pd.read_csv('fiuba_2_postulantes_genero_y_edad_h15.csv')
vistas_df = pd.read_csv('fiuba_3_vistas_h15.csv')
postulaciones_df = pd.read_csv('fiuba_4_postulaciones_h15.csv')
avisos_online_df = pd.read_csv('fiuba_5_avisos_online_h15.csv')
avisos_detalle_df = pd.read_csv('fiuba_6_avisos_detalle_h15.csv')

In [3]:
#Originales TP1 (para entrenar)

postulantes_edu_df2 = pd.read_csv('fiuba_1_postulantes_educacion.csv')
postulantes_gye_df2 = pd.read_csv('fiuba_2_postulantes_genero_y_edad.csv')
vistas_df2 = pd.read_csv('fiuba_3_vistas.csv')
postulaciones_df2 = pd.read_csv('fiuba_4_postulaciones.csv')
avisos_online_df2 = pd.read_csv('fiuba_5_avisos_online.csv')
avisos_detalle_df2 = pd.read_csv('fiuba_6_avisos_detalle.csv')

In [4]:
#Realizo el Merge del tipo outer de los datos
#Originales TP1 y Datos hasta 15 de abril

postulantes_edu_df = postulantes_edu_df.merge(postulantes_edu_df2,how='outer')
postulantes_gye_df = postulantes_gye_df.merge(postulantes_gye_df2,how='outer')
vistas_df = vistas_df.merge(vistas_df2,how='outer')
postulaciones_df = postulaciones_df.merge(postulaciones_df2,how='outer')
avisos_online_df = avisos_online_df.merge(avisos_online_df2,how='outer')
avisos_detalle_df = avisos_detalle_df.merge(avisos_detalle_df2,how='outer')

In [5]:
#Desde 15 de abril (para entrenar)

postulantes_edu_df2 = pd.read_csv('fiuba_1_postulantes_educacion_d15.csv')
postulantes_gye_df2 = pd.read_csv('fiuba_2_postulantes_genero_y_edad_d15.csv')
vistas_df2 = pd.read_csv('fiuba_3_vistas_d15.csv')
avisos_detalle_df2 = pd.read_csv('fiuba_6_avisos_detalle_d15.csv')

In [6]:
#Avisos detalle que faltaban
avisos_detalle_missing = pd.read_csv('fiuba_6_avisos_detalle_missing_nivel_laboral.csv')

In [7]:
#Mergeamos todo

postulantes_edu_df = postulantes_edu_df.merge(postulantes_edu_df2,how='outer')
postulantes_gye_df = postulantes_gye_df.merge(postulantes_gye_df2,how='outer')
vistas_df = vistas_df.merge(vistas_df2,how='outer')
avisos_detalle_df = avisos_detalle_df.merge(avisos_detalle_df2,how='outer')
avisos_detalle_df = avisos_detalle_df.merge(avisos_detalle_missing,how='outer')

# Libero Espacio de la Memoria

In [8]:
postulantes_edu_df2 = 0
postulantes_gye_df2 = 0
vistas_df2 = 0
avisos_detalle_df2 = 0

# Filtrado de Postulantes con informacion completa

In [9]:
postulantes_edu_df.head()

,idpostulante,nombre,estado
0,ZjlZ,Master,En Curso
1,NdJl,Posgrado,En Curso
2,5kNq,Otro,En Curso
3,8rYD,Master,En Curso
4,1Wvj,Universitario,En Curso


In [10]:
postulantes_gye_df.head()

,idpostulante,fechanacimiento,sexo
0,6MM,1985-01-01,MASC
1,Nzz,NaN,NO_DECLARA
2,ZX1,NaN,NO_DECLARA
3,Nq5,NaN,NO_DECLARA
4,ebE,1952-07-07,MASC


In [11]:
print(postulantes_edu_df.shape)
print(postulantes_gye_df.shape)

(693641, 3)
(505382, 3)


In [12]:
print(postulantes_edu_df.idpostulante.unique().shape)
print(postulantes_gye_df.idpostulante.unique().shape)

(447909,)
(504407,)


In [13]:
#Ojo porque hay que ver el tema del secundario y eso. 
#Para la informacion faltante agregamos una Categoria no declara
postulantes_filtrados_df = postulantes_gye_df.merge(postulantes_edu_df,on='idpostulante',how='outer')

print(postulantes_filtrados_df.shape)

postulantes_filtrados_df['idpostulante'].unique().shape

(751563, 5)


(504407,)

In [14]:
postulantes_filtrados_df.head()

,idpostulante,fechanacimiento,sexo,nombre,estado
0,6MM,1985-01-01,MASC,NaN,NaN
1,Nzz,NaN,NO_DECLARA,NaN,NaN
2,ZX1,NaN,NO_DECLARA,NaN,NaN
3,Nq5,NaN,NO_DECLARA,NaN,NaN
4,ebE,1952-07-07,MASC,Universitario,Graduado


In [15]:
postulantes_filtrados_df['nombre'].fillna('NO_DECLARA',inplace=True)
postulantes_filtrados_df['estado'].fillna('NO_DECLARA', inplace=True)
postulantes_filtrados_df['fechanacimiento'].fillna('1918-01-01', inplace=True)

In [16]:
postulantes_filtrados_df.head()

,idpostulante,fechanacimiento,sexo,nombre,estado
0,6MM,1985-01-01,MASC,NO_DECLARA,NO_DECLARA
1,Nzz,1918-01-01,NO_DECLARA,NO_DECLARA,NO_DECLARA
2,ZX1,1918-01-01,NO_DECLARA,NO_DECLARA,NO_DECLARA
3,Nq5,1918-01-01,NO_DECLARA,NO_DECLARA,NO_DECLARA
4,ebE,1952-07-07,MASC,Universitario,Graduado


In [17]:
postulantes_filtrados_df['nombre'].value_counts()

Universitario        251312
Secundario           245184
Terciario/Técnico    113627
NO_DECLARA            56588
Otro                  53466
Posgrado              20671
Master                10092
Doctorado               623
Name: nombre, dtype: int64

In [18]:
#Sacamos los repetidos de postulantes_filtrados_df
#Aca NO se tuvo en cuenta si me quedo con el usuario que hizo Master o Secundario, etc.

postulantes_filtrados_df = postulantes_filtrados_df.drop_duplicates('idpostulante')
postulantes_filtrados_df.shape

(504407, 5)

In [19]:
postulantes_filtrados_df['estado'].value_counts()

Graduado      257790
En Curso      169432
NO_DECLARA     56498
Abandonado     20687
Name: estado, dtype: int64

In [20]:
#Convierto las variables categoricas a variables numericas para poder procesarlas con el algoritmo.
postulantes_filtrados_df['sexo'] = pd.Categorical(postulantes_filtrados_df['sexo'])
postulantes_filtrados_df['sexo'] = postulantes_filtrados_df['sexo'].cat.codes
postulantes_filtrados_df['sexo'].value_counts()

1    250962
2    227653
3     25785
0         7
Name: sexo, dtype: int64

In [21]:
#Convierto las variables categoricas a variables numericas para poder procesarlas con el algoritmo.
postulantes_filtrados_df['nombre'] = pd.Categorical(postulantes_filtrados_df['nombre'])
postulantes_filtrados_df['nombre'] = postulantes_filtrados_df['nombre'].cat.codes
postulantes_filtrados_df['nombre'].value_counts()

7    189432
5    164472
6     66101
2     56498
3     14344
4      7939
1      5286
0       335
Name: nombre, dtype: int64

In [22]:
#Convierto las variables categoricas a variables numericas para poder procesarlas con el algoritmo.
postulantes_filtrados_df['estado'] = pd.Categorical(postulantes_filtrados_df['estado'])
postulantes_filtrados_df['estado'] = postulantes_filtrados_df['estado'].cat.codes
postulantes_filtrados_df['estado'].value_counts()

2    257790
1    169432
3     56498
0     20687
Name: estado, dtype: int64

In [23]:
postulantes_filtrados_df.shape

(504407, 5)

In [24]:
postulantes_filtrados_df.idpostulante.unique().shape

(504407,)

In [25]:
postulantes_filtrados_df.head()

,idpostulante,fechanacimiento,sexo,nombre,estado
0,6MM,1985-01-01,2,2,3
1,Nzz,1918-01-01,3,2,3
2,ZX1,1918-01-01,3,2,3
3,Nq5,1918-01-01,3,2,3
4,ebE,1952-07-07,2,7,2


In [26]:
postulantes_filtrados_df.fechanacimiento.describe()

count         504407
unique         17244
top       1918-01-01
freq           26523
Name: fechanacimiento, dtype: object

In [27]:
#Obtengo la edad
postulantes_edad_trucha = postulantes_filtrados_df[ (postulantes_filtrados_df['fechanacimiento'].str.startswith('0')) == True]
postulantes_edad_trucha['fechanacimiento'] = '1918-01-01'
postulantes_edad_trucha.head()


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,idpostulante,fechanacimiento,sexo,nombre,estado
60580,EWbKXN,1918-01-01,2,7,1
82866,5KK95Z,1918-01-01,2,6,2
91910,680m54,1918-01-01,1,5,2
92659,6wJ8Px,1918-01-01,1,5,2
117876,lDXXla1,1918-01-01,1,5,2


In [31]:
postulantes_filtrados_df = postulantes_filtrados_df[ (postulantes_filtrados_df['fechanacimiento'].str.startswith('0')) == False]
postulantes_filtrados_df = postulantes_filtrados_df.append(postulantes_edad_trucha)
postulantes_filtrados_df.head()


,idpostulante,fechanacimiento,sexo,nombre,estado
0,6MM,1985-01-01,2,2,3
1,Nzz,1918-01-01,3,2,3
2,ZX1,1918-01-01,3,2,3
3,Nq5,1918-01-01,3,2,3
4,ebE,1952-07-07,2,7,2


In [32]:
postulantes_filtrados_df.shape

(504407, 5)

In [34]:
postulantes_filtrados_df.loc[:,'fechanacimiento'] = pd.to_datetime(postulantes_filtrados_df['fechanacimiento']) 
postulantes_filtrados_df.loc[:,'edad'] = (2018 - postulantes_filtrados_df.fechanacimiento.dt.year)
postulantes_filtrados_df.drop(columns='fechanacimiento',axis=1,inplace=True)
postulantes_filtrados_df.head()
#Generar edad por bloques, como queremos hacer con las empresas mas populares.

KeyError: 'fechanacimiento'

# Avisos Detalle

In [35]:
avisos_detalle_df.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,1,Responsable de Marketing-Producto Div. Constru...,<p>Buscamos un <strong>Responsable de Producto...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,1,Asesor de Servicios - Postventa Chevrolet,"<p><span style="""">Para nuestro cliente, conces...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,1,Analistas de Impuestos Sr Ssr y Jr,<p>Importante Estudio de Auditoria y Consultor...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart
3,1112146010,1,Software Engineer,<p> </p><p>Chevron is a multinational company ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,1,Business Developer Manager,"<p>Importante empresa de energías renovables, ...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD


In [36]:
print(avisos_detalle_df.shape)
print(avisos_detalle_df['idaviso'].unique().shape)

(25901, 11)
(25288,)


In [37]:
#Sacamos los repetidos de avisos_detalle
avisos_detalle_filtrados_df = avisos_detalle_df.drop_duplicates('idaviso')
avisos_detalle_df.shape

(25901, 11)

In [38]:
#Me quedo con los avisos del archivo de postulaciones. Esto seria, solo los avisos que tuvieron postulaciones.


#¿Aca me podria quedar con los archivos que NO tuvieron postulaciones para armar un set de NO postulaciones?

#column = postulaciones_df['idaviso'].to_frame()
#avisos_detalle_filtrados_df = column.drop_duplicates('idaviso').merge(avisos_detalle_df,on='idaviso',how='inner')
#avisos_no_postulaciones_df = pd.merge(avisos_detalle_df,avisos_detalle_filtrados_df,on='idaviso',how='right') 
#IMPLEMENTAR
avisos_detalle_filtrados_df.shape

(25288, 11)

In [39]:
#avisos_no_postulaciones_df.shape


In [40]:
#avisos_detalle_df.shape

Hago la conversion de variables categoricas a variables numericas

In [41]:
#Hago un drop de las columnas que no voy a utilizar.
#Mas adelanto debo modificar esto porque la idea es usar denominacion empresa, descripcion y titulo para nuevos features.

avisos_detalle_filtrados_df= avisos_detalle_filtrados_df.drop(columns=['denominacion_empresa','idpais','mapacalle','ciudad','descripcion','titulo'],axis=1)


In [42]:
avisos_detalle_filtrados_df['nivel_laboral'] = pd.Categorical(avisos_detalle_filtrados_df['nivel_laboral'])
avisos_detalle_filtrados_df['nivel_laboral'] = avisos_detalle_filtrados_df['nivel_laboral'].cat.codes
avisos_detalle_filtrados_df['nivel_laboral'].value_counts()

 4    16980
 2     4149
 3     1975
 1     1527
-1      335
 0      322
Name: nivel_laboral, dtype: int64

In [43]:
avisos_detalle_filtrados_df['nombre_area'] = pd.Categorical(avisos_detalle_filtrados_df['nombre_area'])
avisos_detalle_filtrados_df['nombre_area'] = avisos_detalle_filtrados_df['nombre_area'].cat.codes
avisos_detalle_filtrados_df['nombre_area'].value_counts()

185    3153
30     1776
2      1706
143    1538
145     958
42      782
170     759
14      657
118     539
154     424
117     399
82      393
26      392
168     381
5       380
131     376
120     320
153     307
119     295
34      288
4       286
181     278
174     275
137     269
79      261
175     244
46      236
160     234
115     232
158     220
       ... 
16        4
97        3
113       3
10        3
124       3
180       3
29        3
150       3
178       3
179       3
173       3
130       2
73        2
69        2
85        2
107       2
66        2
126       2
108       2
183       2
19        2
56        2
76        1
81        1
70        1
17        1
52        1
149       1
48        1
121       1
Name: nombre_area, Length: 188, dtype: int64

In [44]:
avisos_detalle_filtrados_df['nombre_area'] = pd.Categorical(avisos_detalle_filtrados_df['nombre_area'])
avisos_detalle_filtrados_df['nombre_area'] = avisos_detalle_filtrados_df['nombre_area'].cat.codes
avisos_detalle_filtrados_df['nombre_area'].value_counts()

185    3153
30     1776
2      1706
143    1538
145     958
42      782
170     759
14      657
118     539
154     424
117     399
82      393
26      392
168     381
5       380
131     376
120     320
153     307
119     295
34      288
4       286
181     278
174     275
137     269
79      261
175     244
46      236
160     234
115     232
158     220
       ... 
16        4
97        3
113       3
10        3
124       3
180       3
29        3
150       3
178       3
179       3
173       3
130       2
73        2
69        2
85        2
107       2
66        2
126       2
108       2
183       2
19        2
56        2
76        1
81        1
70        1
17        1
52        1
149       1
48        1
121       1
Name: nombre_area, Length: 188, dtype: int64

In [45]:
avisos_detalle_filtrados_df['tipo_de_trabajo'] = pd.Categorical(avisos_detalle_filtrados_df['tipo_de_trabajo'])
avisos_detalle_filtrados_df['tipo_de_trabajo'] = avisos_detalle_filtrados_df['tipo_de_trabajo'].cat.codes
avisos_detalle_filtrados_df['tipo_de_trabajo'].value_counts()

1    22831
2     1746
7      248
5      125
3      119
8       96
4       88
0       28
6        6
9        1
Name: tipo_de_trabajo, dtype: int64

In [46]:
avisos_detalle_filtrados_df['nombre_zona'] = pd.Categorical(avisos_detalle_filtrados_df['nombre_zona'])
avisos_detalle_filtrados_df['nombre_zona'] = avisos_detalle_filtrados_df['nombre_zona'].cat.codes
avisos_detalle_filtrados_df['nombre_zona'].value_counts()

7     23018
1      2182
0        57
6        10
8         4
9         3
3         3
11        2
4         2
15        1
14        1
13        1
12        1
10        1
5         1
2         1
Name: nombre_zona, dtype: int64

In [47]:
avisos_detalle_filtrados_df.head()

,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area
0,1111556097,7,1,4,51
1,1111949392,7,1,4,90
2,1112145935,7,1,4,86
3,1112146010,7,1,4,145
4,1112211475,7,1,4,30


# Postulaciones

In [48]:
postulantes_filtrados_df['idpostulante'].unique().shape

(504407,)

In [49]:
postulaciones_df.head()

,idaviso,idpostulante,fechapostulacion
0,1112248724,NjlD,2018-01-19 07:39:16
1,1112286523,ZaO5,2018-01-24 15:07:39
2,1112272060,ZaO5,2018-01-24 15:20:10
3,1112288401,ZaO5,2018-01-26 08:37:04
4,1112300563,ZaO5,2018-01-30 13:35:48


In [50]:
#Comparamos los "usuarios" con los  postulantes.

print(postulantes_filtrados_df.idpostulante.unique().shape)
print(postulaciones_df.idpostulante.unique().shape)

(504407,)
(348883,)


In [51]:
#Como resultado tenemos los postulantes que coinciden en postulaciones y postulantes
#Observamos que hay postulaciones de usuarios que no estan en el dataframe de postulantes.
postulantes_filtrados_efectivos_df = postulaciones_df.merge(postulantes_filtrados_df,on='idpostulante',how='inner')
postulantes_filtrados_efectivos_df.idpostulante.unique().shape

(348883,)

In [52]:
postulantes_filtrados_efectivos_df = postulantes_filtrados_efectivos_df.drop_duplicates('idpostulante')
postulantes_filtrados_efectivos_df.shape

(348883, 7)

In [53]:
postulantes_filtrados_efectivos_df =  postulantes_filtrados_efectivos_df.drop(['idaviso','fechapostulacion'],axis='columns')
postulantes_filtrados_efectivos_df = postulantes_filtrados_efectivos_df.reset_index()
postulantes_filtrados_efectivos_df =  postulantes_filtrados_efectivos_df.drop('index',axis='columns')
postulantes_filtrados_efectivos_df.head()

,idpostulante,sexo,nombre,estado,edad
0,NjlD,1,7,2,37
1,ZaO5,1,6,2,40
2,NdJl,2,4,1,49
3,5kNq,2,3,1,45
4,Ez8J,2,7,0,50


In [54]:
postulantes_filtrados_efectivos_df['nombre'].value_counts()

7    136293
5    120231
6     47565
2     25196
3     10633
4      5280
1      3454
0       231
Name: nombre, dtype: int64

In [55]:
# postulantes_filtrados_efectivos_df tiene los postulantes que efectivamente se postularon
# y que ademas tienen informacion sobre edad y estudios, etc.
# Los que estan en postulaciones pero NO en gye y edu, fueron descartados en este paso.
postulantes_filtrados_efectivos_df.head()

,idpostulante,sexo,nombre,estado,edad
0,NjlD,1,7,2,37
1,ZaO5,1,6,2,40
2,NdJl,2,4,1,49
3,5kNq,2,3,1,45
4,Ez8J,2,7,0,50


In [56]:
postulantes_filtrados_efectivos_df.shape

(348883, 5)

# Me fijo si las vistas implican una no postulacion

In [57]:
vistas_df.head()

,idAviso,timestamp,idpostulante
0,1112400775,2018-04-06T01:43:47.445-0400,kPjr2K0
1,1112400775,2018-04-06T01:43:48.330-0400,kPjr2K0
2,1112426664,2018-04-06T01:43:50.042-0400,GNOwZmb
3,1112426664,2018-04-06T01:43:51.171-0400,GNOwZmb
4,1112445738,2018-04-06T01:43:46.371-0400,lDLxajz


In [58]:
vistas_df.columns=['idaviso','timestamp','idpostulante']

In [59]:
postulaciones_df.head()

,idaviso,idpostulante,fechapostulacion
0,1112248724,NjlD,2018-01-19 07:39:16
1,1112286523,ZaO5,2018-01-24 15:07:39
2,1112272060,ZaO5,2018-01-24 15:20:10
3,1112288401,ZaO5,2018-01-26 08:37:04
4,1112300563,ZaO5,2018-01-30 13:35:48


In [60]:
#Efectivamente una "vista" no implica una no postulacion.
postulaciones_df.merge(vistas_df,on=['idaviso','idpostulante'],how='inner').head()

,idaviso,idpostulante,fechapostulacion,timestamp
0,1112364763,ZaO5,2018-02-27 20:54:50,2018-02-27T20:53:53.668-0500
1,1112383420,ZaO5,2018-03-07 08:16:46,2018-04-02T18:38:56.797-0400
2,1112383420,ZaO5,2018-03-07 08:16:46,2018-04-02T18:39:20.453-0400
3,1112397910,ZaO5,2018-03-18 15:35:40,2018-04-02T18:42:17.247-0400
4,1112397910,ZaO5,2018-03-18 15:35:40,2018-04-02T18:44:18.643-0400


# Armado del Set de Entrenamiento

In [61]:
# Primero armo un dataframe con la informacion de postulaciones que ya tengo -> postulantes_df
# Observar que aca puede haber postulantes sobre los cuales no tengo informacion sobre genero y edad.
# Tambien puede haber postulantes que luego no aparezcan en el test final.


# ¿Deberia filtrar los postulantes sobre los cuales no tengo info sobre su genero y edad?
avisos = avisos_detalle_filtrados_df
avisos = avisos['idaviso'].to_frame()

postulantes = postulantes_filtrados_df.drop_duplicates('idpostulante')
postulantes = postulantes['idpostulante'].to_frame()


postulantes_df =  postulaciones_df.merge(avisos,how='inner',on='idaviso')
postulantes_df = postulantes_df.merge(postulantes,how='inner',on='idpostulante')

postulantes_df =postulantes_df.groupby('idpostulante').agg({'idaviso':'value_counts'})
postulantes_df.columns=['postulacion']
postulantes_df['postulacion'] = 1 #Para los casos en que se repiten.

avisos = 0
postulantes = 0

In [62]:
postulantes_df.head()

postulacion
idpostulante idaviso                
0z5Dmrd      1112384041            1
             1112420060            1
0z5JW1r      1112257171            1
             1112281879            1
             1112292169            1

In [63]:
postulantes_df.reset_index(inplace=True)

In [64]:
postulantes_df.head()

,idpostulante,idaviso,postulacion
0,0z5Dmrd,1112384041,1
1,0z5Dmrd,1112420060,1
2,0z5JW1r,1112257171,1
3,0z5JW1r,1112281879,1
4,0z5JW1r,1112292169,1


In [65]:
postulantes_df.idaviso.unique().shape

(18695,)

In [66]:
avisos_detalle_df['idaviso'].unique().shape

(25288,)

In [67]:
#Armado de NO postulaciones

#Verificar que las listas utilizadas sean las correctas. Estamos usando solo avisos con postulacion.

#¿Podriamos usar los avisos que no tienen postulacion?


lista = []
user_list = postulantes_filtrados_df['idpostulante'].unique() #Utilizo a todos los usuarios. 
aviso_list = avisos_detalle_filtrados_df['idaviso'].unique() #Utilizo todos los avisos.
index = 1
for n in range(10000):
    for i in range(500):
        user_index = random.randint(0,504371)
        aviso_index = random.randint(0,25287)
        lista.append((index,user_list[user_index],aviso_list[aviso_index],0))
        index+=1
        

In [68]:
no_postulantes_df = pd.DataFrame(lista)

In [69]:
#postulantes_filtrados_df=0
#avisos_detalle_df= 0
#avisos_detalle_filtrados_df=0
#avisos_online_df=0
#postulaciones_df=0
lista=0

In [70]:
no_postulantes_df.memory_usage(deep=True).sum()

438943080

In [71]:
no_postulantes_df.head()

,0,1,2,3
0,1,6rQN41l,1112410114,0
1,2,jkM8ppO,1112419965,0
2,3,LNPGBZb,1112268735,0
3,4,mzGWG66,1112393719,0
4,5,jkV95rj,1112312606,0


In [72]:
no_postulantes_df.drop(0,axis='columns',inplace=True)

In [73]:
no_postulantes_df.head()

,1,2,3
0,6rQN41l,1112410114,0
1,jkM8ppO,1112419965,0
2,LNPGBZb,1112268735,0
3,mzGWG66,1112393719,0
4,jkV95rj,1112312606,0


In [74]:
no_postulantes_df.columns= ['idpostulante','idaviso','postulacion']

In [75]:
no_postulantes_df.shape

(5000000, 3)

In [76]:
no_postulantes_df.head()

,idpostulante,idaviso,postulacion
0,6rQN41l,1112410114,0
1,jkM8ppO,1112419965,0
2,LNPGBZb,1112268735,0
3,mzGWG66,1112393719,0
4,jkV95rj,1112312606,0


In [77]:
postulantes_df.head()

,idpostulante,idaviso,postulacion
0,0z5Dmrd,1112384041,1
1,0z5Dmrd,1112420060,1
2,0z5JW1r,1112257171,1
3,0z5JW1r,1112281879,1
4,0z5JW1r,1112292169,1


In [78]:
#Junto el dataframe donde tengo los postulantes con el aviso al que se postularon
#Con el dataframe generado al azar.
entrenamiento_df = no_postulantes_df.merge(postulantes_df,how='outer',on=['idpostulante','idaviso'])

In [79]:
entrenamiento_df.head()

,idpostulante,idaviso,postulacion_x,postulacion_y
0,6rQN41l,1112410114,0.0,NaN
1,jkM8ppO,1112419965,0.0,NaN
2,LNPGBZb,1112268735,0.0,NaN
3,mzGWG66,1112393719,0.0,NaN
4,jkV95rj,1112312606,0.0,NaN


In [80]:
entrenamiento_df['postulacion_x'].fillna(0,inplace=True)
entrenamiento_df['postulacion_y'].fillna(0,inplace=True)

In [81]:
entrenamiento_df['postulacion'] = entrenamiento_df['postulacion_x'] + entrenamiento_df['postulacion_y']

In [82]:
entrenamiento_df.head()

,idpostulante,idaviso,postulacion_x,postulacion_y,postulacion
0,6rQN41l,1112410114,0.0,0.0,0.0
1,jkM8ppO,1112419965,0.0,0.0,0.0
2,LNPGBZb,1112268735,0.0,0.0,0.0
3,mzGWG66,1112393719,0.0,0.0,0.0
4,jkV95rj,1112312606,0.0,0.0,0.0


In [83]:
 entrenamiento_df.drop(['postulacion_x','postulacion_y'],axis='columns',inplace=True)

In [84]:
entrenamiento_df['postulacion'] = pd.Categorical(entrenamiento_df['postulacion'])
entrenamiento_df['postulacion'] = entrenamiento_df['postulacion'].cat.codes
entrenamiento_df['postulacion'].value_counts()

1    6187929
0    4997616
Name: postulacion, dtype: int64

In [85]:
# Hasta aca Ya tenemos nuestro formato (id,postulante,aviso,postulacion)
entrenamiento_df.head()

,idpostulante,idaviso,postulacion
0,6rQN41l,1112410114,0
1,jkM8ppO,1112419965,0
2,LNPGBZb,1112268735,0
3,mzGWG66,1112393719,0
4,jkV95rj,1112312606,0


In [86]:
entrenamiento_df.tail()

,idpostulante,idaviso,postulacion
11185540,zvxzrWm,1112438823,1
11185541,zvxzrWm,1112442213,1
11185542,zvxzrWm,1112445486,1
11185543,zvxzrWm,1112448822,1
11185544,zvxzrWm,1112455515,1


In [87]:
entrenamiento_df.idaviso.unique().shape

(25288,)

In [88]:
avisos_detalle_df.idaviso.unique().shape

(25288,)

# Armo la Matriz de Features

In [89]:
matrix_df = entrenamiento_df.merge(avisos_detalle_filtrados_df,how='inner',on='idaviso')

In [90]:
matrix_df.idaviso.shape

(11185545,)

In [91]:
#Deberian ser iguales.
matrix_df.shape[0] == entrenamiento_df.shape[0]

True

In [92]:
matrix_df = matrix_df.merge(postulantes_filtrados_df,how='inner',on='idpostulante')

In [93]:
#Deberian ser iguales.
matrix_df.shape[0] == entrenamiento_df.shape[0]

True

In [94]:
#Me quedo con el indice como "id" de la tupla postulante-aviso.
matrix_df.drop(columns=['idpostulante','idaviso'],axis=1,inplace=True)

In [95]:
matrix_df.head()

,postulacion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,sexo,nombre,estado,edad
0,0,7,1,4,42,2,5,2,23
1,0,7,1,4,115,2,5,2,23
2,0,7,1,4,30,2,5,2,23
3,0,7,1,2,2,2,5,2,23
4,0,7,1,4,30,2,5,2,23


In [96]:
#Normalizar la matriz

# Test Final DF

In [140]:
test_final_df = pd.read_csv('test_final_100k.csv')

In [98]:
test_final_df.head()

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL
2,2,739260,ezRKm9
3,3,758580,1Q35ej
4,4,758580,EAN4J6


In [99]:
test_final_df['idpostulante'].unique().shape

(64646,)

In [100]:
#Los postulantes que tengo informacion que aparecen en el test final.
#Entonces hay casi 20mil postulantes sobre los cuales no tengo informacion.
test_final_df.drop_duplicates('idpostulante').merge(postulantes_filtrados_df,on='idpostulante',how='inner').shape

(64646, 7)

# KNN

In [101]:
knn = KNeighborsClassifier(n_neighbors= 5)

In [102]:
entrenamiento_df.head()

,idpostulante,idaviso,postulacion
0,6rQN41l,1112410114,0
1,jkM8ppO,1112419965,0
2,LNPGBZb,1112268735,0
3,mzGWG66,1112393719,0
4,jkV95rj,1112312606,0


In [103]:
matrix_df.head()

,postulacion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,sexo,nombre,estado,edad
0,0,7,1,4,42,2,5,2,23
1,0,7,1,4,115,2,5,2,23
2,0,7,1,4,30,2,5,2,23
3,0,7,1,2,2,2,5,2,23
4,0,7,1,4,30,2,5,2,23


In [104]:
X = matrix_df[matrix_df['postulacion'] == 0].head(300000).append(matrix_df[matrix_df['postulacion'] == 1].head(200000))

In [105]:
X.shape

(500000, 9)

In [106]:
y = X['postulacion']
X = X.drop('postulacion',axis=1)

In [107]:
X.head()

,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,sexo,nombre,estado,edad
0,7,1,4,42,2,5,2,23
1,7,1,4,115,2,5,2,23
2,7,1,4,30,2,5,2,23
3,7,1,2,2,2,5,2,23
4,7,1,4,30,2,5,2,23


In [108]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: postulacion, dtype: int8

In [109]:
entrenamiento_df = ""
postulantes_df = ""
no_postulantes_df = ""

In [110]:
y.value_counts()

0    300000
1    200000
Name: postulacion, dtype: int64

In [111]:
knn.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [126]:
test_final_df = test_final_df.merge(avisos_detalle_filtrados_df,on='idaviso',how='left')

In [127]:
test_final_df.head()

,id,idaviso,idpostulante,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area
0,0,739260,6M9ZQR,1,1,1,30
1,1,739260,6v1xdL,1,1,1,30
2,2,739260,ezRKm9,1,1,1,30
3,3,758580,1Q35ej,1,1,3,170
4,4,758580,EAN4J6,1,1,3,170


In [128]:
test_final_df.shape

(100000, 7)

In [129]:
test_final_df = test_final_df.merge(postulantes_filtrados_df,on='idpostulante',how='inner')

In [130]:
test_final_df.shape

(100000, 11)

In [131]:
test_final_df.head()

,id,idaviso,idpostulante,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,sexo,nombre,estado,edad
0,0,739260,6M9ZQR,1,1,1,30,1,0,2,42
1,1,739260,6v1xdL,1,1,1,30,2,3,1,31
2,2,739260,ezRKm9,1,1,1,30,1,7,2,36
3,3,758580,1Q35ej,1,1,3,170,2,4,2,69
4,4,758580,EAN4J6,1,1,3,170,1,6,1,32


In [132]:
test_final_df = test_final_df.drop(['id','idaviso','idpostulante'],axis=1)

In [133]:
test_final_df.head()

,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,sexo,nombre,estado,edad
0,1,1,1,30,1,0,2,42
1,1,1,1,30,2,3,1,31
2,1,1,1,30,1,7,2,36
3,1,1,3,170,2,4,2,69
4,1,1,3,170,1,6,1,32


In [134]:
test_final_df.shape

(100000, 8)

In [135]:
X.shape

(500000, 8)

In [136]:
X.head()

,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,sexo,nombre,estado,edad
0,7,1,4,42,2,5,2,23
1,7,1,4,115,2,5,2,23
2,7,1,4,30,2,5,2,23
3,7,1,2,2,2,5,2,23
4,7,1,4,30,2,5,2,23


In [137]:
pred = knn.predict(test_final_df)

In [139]:
pred

array([0, 1, 0, ..., 1, 1, 0], dtype=int8)

In [142]:
test_final_df.head()

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL
2,2,739260,ezRKm9
3,3,758580,1Q35ej
4,4,758580,EAN4J6


In [144]:
test_final_df['sepostulo'] = pd.DataFrame(pred)

In [145]:
test_final_df.head()

,id,idaviso,idpostulante,sepostulo
0,0,739260,6M9ZQR,0
1,1,739260,6v1xdL,1
2,2,739260,ezRKm9,0
3,3,758580,1Q35ej,0
4,4,758580,EAN4J6,0


In [146]:
submit = test_final_df.drop(['idaviso','idpostulante'],axis=1)

In [147]:
submit.head()

,id,sepostulo
0,0,0
1,1,1
2,2,0
3,3,0
4,4,0


In [148]:
submit.sepostulo.value_counts()

0    68846
1    31154
Name: sepostulo, dtype: int64

In [152]:
submit.to_csv('Submit_file')